In [10]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision.transforms import ToTensor
import cv2
from tqdm import tqdm
from torchvision import models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


# 데이터셋 경로
noisy_data_path = 'C:/Users/p8528/Desktop/test_search/Uni-D-Datathon-4th/test'


class CustomDataset(Dataset):
    def __init__(self, noisy_image_paths, transform=None):
        # 주어진 디렉토리 내의 이미지 파일 경로 리스트를 생성합니다.
        self.noisy_image_paths = [
            os.path.join(noisy_image_paths, x) for x in os.listdir(noisy_image_paths) 
            if x.endswith(('jpg', 'png', 'jpeg'))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.noisy_image_paths)

    def __getitem__(self, index):
        # 이미지 경로 가져오기
        noisy_image_path = self.noisy_image_paths[index]
        
        # 이미지 불러오기
        noisy_image = Image.open(noisy_image_path).convert("RGB")
        
        # 전처리가 이미 되어 있으므로 추가 변환은 필요하지 않지만, transform이 있을 경우 적용합니다.
        if self.transform:
            noisy_image = self.transform(noisy_image)
        
        return noisy_image, noisy_image_path  # 정답 레이블 없이 이미지와 경로만 반환

# transform 예시 (필요 시에만 사용)
transform = ToTensor()

# 데이터셋 사용 예시
test_dataset = CustomDataset(noisy_image_paths=noisy_data_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
from torchvision import models

# Device 설정 (CUDA가 가능한 경우 CUDA 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 설정 및 가중치 로드
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 20)  # 클래스 수에 맞게 출력 설정
model = model.to(device)  # 모델을 CUDA로 이동
model.load_state_dict(torch.load('resnet18_final.pth'))
model.eval()  # 평가 모드로 설정

predicted_results = []  # 예측 결과 저장할 리스트

# 예측 수행
with torch.no_grad():
    for images, image_paths in tqdm(test_loader, desc="Predicting"):
        images = images.to(device)  # 이미지를 CUDA로 이동
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        # 이미지 경로와 예측된 라벨 저장
        for i in range(len(predicted)):
            predicted_results.append([
                image_paths[i],
                predicted[i].item()
            ])

# 예측 결과 확인
for result in predicted_results[:5]:  # 상위 5개 결과 미리보기
    print(f"Image path: {result[0]}, Predicted label: {result[1]}")


c:\Users\p8528\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\p8528\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\p8528\AppData\Local\Temp\ipykernel_8912\3112979124.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickl